# Day 8: Handheld Halting

https://adventofcode.com/2020/day/8

## Part 1

### Op details (note to self)

- `acc` increases or decreases a single global value called the accumulator by the value given in the argument. For example, `acc +7` would increase the accumulator by 7. The accumulator starts at 0. After an acc instruction, the instruction immediately below it is executed next.
- `jmp` jumps to a new instruction relative to itself. The next instruction to execute is found using the argument as an offset from the jmp instruction; for example, `jmp +2` would skip the next instruction, `jmp +1` would continue to the instruction immediately below it, and `jmp -20` would cause the instruction 20 lines above to be executed next.
- `nop` stands for **N**o **OP**eration - it does nothing. The instruction immediately below it is executed next.

In [22]:
# Grab our data
from pathlib import Path

INPUTS = Path("input.txt").resolve().read_text().strip()

instructions = [x.strip() for x in INPUTS.split("\n")]

### Details

So, we need to run consecutively through the commands, starting with `acc = 0`, and determine:

1. the point at which any command is re-run; and
2. what the value of `acc` is prior to that execution.

Simple enough. We need to keep a running set of command lines that have been executed from the set of instructions, then continuously check if the current command index exists in that set. Meanwhile, we take the appropriate operation given the current command line instruction.

In [25]:
def check_instructions(lines):
    curr = 0
    accum = 0
    seen = {curr}
    valid = True
    while True:
        accum_delta = 0
        try:
            line = lines[curr]
        except IndexError:
            # We hit the end?
            break
        op, arg = line.split()

        if op == "nop":
            delta = 1
        elif op == "acc":
            delta = 1
            accum_delta = int(arg)
        elif op == "jmp":
            delta = int(arg)

        if (curr + delta) in seen:
            valid = False
            break
        accum += accum_delta
        curr += delta
        seen.add(curr)
    return accum, valid


accumulator, _ = check_instructions(instructions)

print(f"Value of acc at the loop point: {accumulator}")

Value of acc at the loop point: 2051


## Part 2

Similar to the first part, but this time need to determine what one instruction - either a `jmp` or `nop` - has to be swapped in order to generate a solution.

I'll admit, I got stumped by this for a bit, then looked around for similar solutions made by others. Where it seems I got stuck was by initially assuming that the one instruction causing the loop in Part 1 was the same instruction that had to be swapped; but that's not necessarily true.

Instead, we need to go through the code lines and swap `jmp`s to `nop`s and vice versa to test which set of full code lines can actually generate a good solution.

In [28]:
for idx, line in enumerate(instructions):
    test_instructions = instructions[:]
    op, arg = line.split()

    # Do the op swap (doo, doo doo duh-doo duh-doo doo DOO!)
    if op == "jmp":
        op = "nop"
    elif op == "nop":
        op = "jmp"

    new_line = f"{op} {arg}"
    test_instructions[idx] = new_line

    accum, valid = check_instructions(test_instructions)
    if valid:
        break

print(f"Valid accumulator is {accum}, with a swapped instruction at line {idx + 1}")

Valid accumulator is 2304, with a swapped instruction at line 322
